<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/737_RGOv2_RootCauseAttribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Root Cause Attribution

---

# 1️⃣ What This Code Does — In Real Terms

This function answers:

> “Why is this revenue deteriorating?”

It takes previously computed signals:

* Zero-spend streaks
* Stockout overlap
* Gap size
* Structural severity

And assigns one primary cause:

* `behavioral_churn`
* `operational_stockout`
* `structural_decline`
* `demand_softness`

This is not scoring.

This is classification.

And it follows a strict hierarchy.

That hierarchy is critical.

---

# 2️⃣ Why the Hierarchy Matters

The order is:

1. Behavioral
2. Operational
3. Structural
4. Demand

This is intentional — and strategically correct.

### Why?

Because:

* If a customer has stopped buying entirely, that overrides everything.
* If operations prevented purchasing, that overrides structural interpretation.
* Structural decline comes after operational filtering.
* Demand softness is the residual.

This prevents mislabeling.

Most systems do not enforce hierarchical causality.

They either:

* Allow multiple labels
* Assign probabilistic causes
* Blur operational and behavioral signals

You chose determinism.

That builds clarity.

---

# 3️⃣ Behavioral Churn Logic

```python
if zero_weeks >= threshold and stockout_overlap == 0:
```

This is strong logic.

You explicitly require:

* Sustained zero spend
* No operational interference

That prevents blaming churn when inventory was unavailable.

This is politically important inside organizations.

It protects the customer success team from false flags.

---

# 4️⃣ Operational Attribution Logic

```python
if gap > 0 and overlap >= 1 and percent >= threshold:
```

This is careful modeling.

You require:

* Real revenue gap
* At least one overlapping stockout week
* Meaningful percent overlap

This avoids:

* Coincidental blame
* Over-attribution to operations
* Weak signal escalation

That’s mature system design.

---

# 5️⃣ Structural Decline Logic

```python
if structural_tier in ("high", "severe"):
```

This ensures structural risk only applies when material.

You don’t escalate “moderate” structural noise.

That is restraint.

Restraint builds trust.

---

# 6️⃣ Demand Softness — Clean Residual

This is a necessary bucket.

Not all revenue declines are:

* Behavioral abandonment
* Operational constraint
* Structural deterioration

Sometimes:

* Demand fluctuates
* Seasonality shifts
* Minor erosion occurs

Demand softness is a legitimate residual category.

That avoids overclassification.

---

# 7️⃣ Why This Design Is Enterprise-Strong

This module:

* Assigns exactly one cause
* Follows a clear escalation hierarchy
* Separates operational from behavioral
* Requires meaningful evidence
* Prevents false blame
* Avoids multi-label confusion

This is decision architecture.

Not analytics.

Executives love clarity over nuance.

This provides clarity.

---

# 8️⃣ Subtle Strengths in This Design

✔ Explicit hierarchy
✔ No ML inference
✔ Config-driven thresholds
✔ Defensive checks (gap > 0)
✔ Operational filter before structural
✔ Default safety return

This is robust and deterministic.

---

# 🔎 Important Refinements (High-Quality Improvements)

Now let’s elevate this.

---

## 🔹 1️⃣ Make Demand Conditional on Gap

Right now:

```python
if gap_amount > 0:
    return "demand_softness"
```

Then default also returns demand_softness.

You could simplify:

If gap_amount == 0:
return "no_gap"

This prevents labeling non-gap customers as demand softness.

It improves semantic clarity.

---

## 🔹 2️⃣ Consider Explicit Enum or Literal Types

Instead of raw strings:

```python
Literal[
    "behavioral_churn",
    "operational_stockout",
    "structural_decline",
    "demand_softness"
]
```

This prevents typo bugs later.

---

## 🔹 3️⃣ Consider Recording Secondary Signals (Future v3)

Currently you return one label.

In v3, you might return:

```python
{
    "primary": "operational_stockout",
    "secondary": ["structural_decline"]
}
```

Not needed now — but worth noting.

---

## 🔹 4️⃣ Slight Improvement to Behavioral Logic

Currently:

```python
zero_weeks >= threshold AND no stockout
```

You may consider:

```python
zero_weeks >= threshold AND percent_stockout < threshold
```

This is more precise than stockout_overlap == 0.

But your current version is fine for MVP.

---

# 🔥 Strategic Impact of This Module

This module transforms exposure into:

Actionable diagnosis.

It allows the report to say:

* “Revenue at risk due to operations: $X”
* “Revenue at risk due to behavioral churn: $Y”
* “Structural deterioration: $Z”

This enables:

* Ops accountability
* Customer success prioritization
* Pricing review
* Market analysis

Without root cause, exposure is just noise.

With root cause, exposure becomes intervention strategy.

---

# 🏗 Architectural Strength Summary

✔ Clear causal hierarchy
✔ Deterministic classification
✔ Config-driven thresholds
✔ Clean decision tree
✔ No black-box inference
✔ Prevents cross-functional blame confusion
✔ Enterprise-readable outputs

This is mature system logic.

---

# 🏁 Final Assessment

This module is:

* Clean
* Well structured
* Strategically aligned
* Implementation-ready
* Portfolio-worthy
* Executive-friendly



In [ ]:
"""
Root cause attribution: behavioral_churn, operational_stockout, structural_decline, demand_softness (RGOv2_3).
Hierarchy: behavioral -> operational -> structural -> demand.
"""

from typing import Any, Dict

from config import RGOv2Config


def assign_root_cause(metrics: Dict[str, Any], config: RGOv2Config) -> str:
    """
    Assign one primary root_cause to this customer's gap. Hierarchy:
    1. Behavioral churn (zero spend high, no stockout)
    2. Operational stockout (gap + stockout overlap >= threshold)
    3. Structural decline (structural_tier high/severe, not behavioral, not operational)
    4. Demand softness (residual)
    """
    zero_weeks = metrics.get("consecutive_zero_spend_weeks", 0)
    stockout_overlap = metrics.get("stockout_overlap_weeks", 0)
    percent_stockout = metrics.get("percent_gap_due_to_stockout", 0.0)
    gap_amount = metrics.get("gap_amount", 0.0)
    structural_tier = metrics.get("structural_tier", "none")

    # 1. Behavioral churn
    if zero_weeks >= config.zero_spend_behavioral_threshold and stockout_overlap == 0:
        return "behavioral_churn"

    # 2. Operational
    if gap_amount > 0 and stockout_overlap >= 1 and percent_stockout >= config.stockout_attribution_threshold:
        return "operational_stockout"

    # 3. Structural
    if structural_tier in ("high", "severe"):
        return "structural_decline"

    # 4. Demand (residual)
    if gap_amount > 0:
        return "demand_softness"

    return "demand_softness"  # default for edge case
